In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen



In [2]:
provincias = pd.read_csv('raw_data/provincias.csv')
energy = pd.read_csv('raw_data/Spain_energy_df_2017.csv')

In [3]:
provincias_grouped = provincias.groupby('Provincia').agg({'latitude':'mean','longitude':'mean'}).reset_index()

In [4]:
def get_closest_province(row):
    df = provincias_grouped[['latitude','longitude']] - list(row[['lat','lon']])

    df['agg'] = df['latitude']**2 + df['longitude']**2
    name_index = df.sort_values('agg').iloc[0,:].name

    return provincias_grouped.iloc[name_index].Provincia
energy['province'] = energy.apply(get_closest_province,axis=1)

energy.groupby('province').agg({'Solar Generation (2017)':'sum'}).sort_values('Solar Generation (2017)',ascending=False).head(3)

,Solar Generation (2017)
province,
Tarragona,29594.68
Caceres,18406.04
Valencia/Val�ncia,15133.59


In [10]:
energy.head(3)

,Unnamed: 0,country,capacity_mw,lat,lon,primary_fuel,Solar Generation (2017),province
0,20135,ESP,877.66,43.5528,-5.7231,Coal,4003.64,Asturias
1,20136,ESP,758.74,39.9427,-3.8548,Gas,1798.46,Toledo
2,20137,ESP,15.00,41.5485,0.8245,Waste,5.22,Lleida


In [5]:
import pydeck as pdk


In [6]:
data = energy

layer = pdk.Layer(
    'HexagonLayer',  # `type` positional argument is here
    data,
    get_position=['lon', 'lat'],
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,
    coverage=1)


# Combined all of it and render a viewport
r = pdk.Deck(layers=[layer])

In [23]:
data = data.rename(columns={'Solar Generation (2017)':'solar_generation'})

In [24]:
data

,Unnamed: 0,country,capacity_mw,lat,lon,primary_fuel,solar_generation,province
0,20135,ESP,877.660,43.5528,-5.7231,Coal,4003.64,Asturias
1,20136,ESP,758.740,39.9427,-3.8548,Gas,1798.46,Toledo
2,20137,ESP,15.000,41.5485,0.8245,Waste,5.22,Lleida
3,20138,ESP,18.000,41.8559,-1.9224,Wind,50.55,Soria
4,20139,ESP,16.334,41.8559,-1.9224,Waste,5.68,Soria
...,...,...,...,...,...,...,...,...
824,20959,ESP,9.000,41.6428,-4.8151,Solar,15.62,Valladolid
825,20960,ESP,7.600,39.8710,-6.8320,Solar,14.92,Caceres
826,20961,ESP,9.000,41.9340,-0.8050,Solar,16.03,Zaragoza
827,20962,ESP,9.900,41.9440,-0.8180,Solar,17.37,Zaragoza


In [54]:
DATA_URL = "https://raw.githubusercontent.com/deldersveld/topojson/master/countries/spain/spain-province.json"


In [57]:
import pydeck as pdk
import pandas as pd
from pydeck.types import String


CPU_GRID_LAYER_DATA = (
    "https://raw.githubusercontent.com/uber-common/" "deck.gl-data/master/website/sf-bike-parking.json"
)
df = pd.read_json(CPU_GRID_LAYER_DATA)

# Define a layer to display on a map

layer = pdk.Layer(
    "GridLayer",
    data,
    pickable=True,
    extruded=True,
    stroked=False,
    filled=True,
    wireframe=True,
    cell_size=15000,
    elevation_scale=200,
    get_position=['lon', 'lat'],
)

layer2 = pdk.Layer(
    "HeatmapLayer",
    data,
    opacity=0.5,
    get_position=['lon', 'lat'],
    aggregation=String('SUM'),
    get_weight="solar_generation")




view_state = pdk.ViewState(latitude=43.5528	, longitude=-5.7231, zoom=3, bearing=0, pitch=45)

# Render
r = pdk.Deck(
    layers=[layer,layer2],
    initial_view_state=view_state,
    tooltip={"text": "{position}\nSolar Generation: {solar_generation}"},
)
r.to_html("grid_layer.html")

In [42]:
energy.loc[energy['province'].str.contains('erife')].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 20 to 767
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               13 non-null     int64  
 1   country                  13 non-null     object 
 2   capacity_mw              13 non-null     float64
 3   lat                      13 non-null     float64
 4   lon                      13 non-null     float64
 5   primary_fuel             13 non-null     object 
 6   Solar Generation (2017)  13 non-null     float64
 7   province                 13 non-null     object 
dtypes: float64(4), int64(1), object(3)
memory usage: 936.0+ bytes
